# Cleaning

In [1]:
import os
import pandas as pd
from tqdm import tqdm

Semua data yang berhasil di-*scrape* disimpan di folder `./raw`

In [2]:
len(os.listdir('raw'))

4846

Buat dataframe dan fungsi untuk menghasilkan+menambahkan data bersih dari setiap file ke dataframe

In [3]:
katalog = pd.DataFrame()

def formatted(filename):
    global katalog
    
    with open(filename, 'r') as f:
        stream = f.read().split('\n')

    table = []
    for s in stream:
        if s=='' or 'href=viewmap.php?' not in s: continue
        
        if 'href=viewfm.php?' in s:
            entry = s.split('href=viewfm.php?')[-1].split('>')[0]
        else:
            entry = s.split('href=viewmap.php?')[-1].split('>')[0]

        record = {}
        for column in entry.split('&'):
            try:
                key, val = column.split('=')
                record[key]=val
            except:
                pass
        table.append(record)

    df = pd.DataFrame(table)
    
    katalog = katalog.append(df)

In [4]:
for fname in tqdm(os.listdir('raw')):
    if fname[-4:]!='html':
        continue

    formatted('raw/'+fname)

100%|██████████| 4846/4846 [05:32<00:00, 14.59it/s]


In [5]:
for column in katalog:
    # mencari kolom dengan kurang dari 30 data unik
    z = set(katalog[column])
    if len(z)>30: continue

    print(column)
    print(z)

# ternyata bisa dihapus....

# katalog['ketlon'] sebenarnya hanya berisi 'E' dan 'W'
# Namun karena geografi Indonesia hanya berisi 'E'
# dan data selain 'E' hanya berisi data kotor... 
katalog = katalog[katalog['ketlon']=='E'] 

# bersihkan formatting data pada kolom remark
clean = lambda v: v.replace('+', ' ').replace('%2C', '-').strip()
katalog['remark'] = katalog['remark'].apply(clean)

# gabungkan data kolom ketlat dan kolom lat
# lat bernilai positif jika ketlat bernilai 'N'
# dan negatif jika ketlat bernilai 'S' 
signed = lambda v: v[0] if v[1]=='N' else '-'+v[0]
katalog['lat'] = katalog[['lat', 'ketlat']].apply(signed, axis=1)

# hapus kolom-kolom yang tidak digunakan
katalog = katalog.drop(columns=['ketlon', 'ketlat', 'status'])

katalog.to_csv('katalog_gempa.tsv', sep='\t', index=False)

ketlat
{'', nan, '9.5', 'S', 'Warning', 'given', 'Lon', '110', 'print', 'until', 'the', '[service://]location', 'N', ']', 'send', 'level', 'supported', 'Use', 'for:', 'alternative'}
ketlon
{'', 'E', 'syslog', 'file', 'from', 'mode:', 'component', 'instead', 'ID', '(InaTEWS)', 'than', 'M', 'path', 'to', 'output', 'verbosity', 'the', 'drivers'}
status
{'', '.....', 'unset'}


In [6]:
katalog.fillna('')

,tgl,ot,lat,lon,depth,mag,remark,strike1,dip1,rake1,strike2,dip2,rake2
0,2010/03/02,23:41:57.804,3.74,127.08,45,4.7,Talaud Islands- Indonesia,,,,,,
1,2010/03/02,23:41:30.530,0.75,99.68,10,3.1,Northern Sumatra- Indonesia,,,,,,
2,2010/03/02,23:23:29.343,0.92,120.09,10,3.5,Minahassa Peninsula- Sulawesi,,,,,,
3,2010/03/02,22:46:07.861,0.35,120.64,10,3.4,Minahassa Peninsula- Sulawesi,,,,,,
4,2010/03/02,22:33:34.591,1.52,97.35,28,3.4,Northern Sumatra- Indonesia,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2010/12/28,16:00:59.742,-5.00,103.12,34,3.9,Southern Sumatra- Indonesia,,,,,,
4,2010/12/28,14:09:03.564,4.47,96.22,10,3.3,Northern Sumatra- Indonesia,,,,,,
5,2010/12/28,06:58:43.983,-7.24,107.44,10,3.4,Java- Indonesia,,,,,,
6,2010/12/28,04:28:40.256,-7.16,107.75,302,3.8,Java- Indonesia,,,,,,
